In [ ]:
import string
import re
import os
import random
from sklearn.utils import shuffle
import numpy as np
import nltk
nltk.download('twitter_samples')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, twitter_samples
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow import keras

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:



import string
import re
import os
import nltk

nltk.download('twitter_samples')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, twitter_samples

tweet_tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

# Stop words are messy and not that compelling;
# "very" and "not" are considered stop words, but they are obviously expressing sentiment

# The porter stemmer lemmatizes "was" to "wa".  Seriously???

# I'm not sure we want to get into stop words
stopwords_english = stopwords.words('english')

# Also have my doubts about stemming...
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()



[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    ### START CODE HERE ###
    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                    word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)
    ### END CODE HERE ###
    return tweets_clean


# let's not reuse variables
# all_positive_tweets = twitter_samples.strings('positive_tweets.json')
# all_negative_tweets = twitter_samples.strings('negative_tweets.json')

def load_tweets():
    all_positive_tweets = twitter_samples.strings('positive_tweets.json')
    all_negative_tweets = twitter_samples.strings('negative_tweets.json')
    return all_positive_tweets, all_negative_tweets

In [ ]:

all_positive_tweets, all_negative_tweets=load_tweets()

print('number of positive tweets',len(all_positive_tweets))
print('number of negative tweets',len(all_negative_tweets))

train_x=all_positive_tweets[:4000]+all_negative_tweets[:4000]

val_x=all_positive_tweets[4000:]+all_negative_tweets[4000:]


train_y=np.array([1]*4000+[0]*4000)
val_y=np.array([1]*1000+[0]*1000)

print('number of training tweets',len(train_x))
print('number of validation tweets',len(val_x))

print('train_y.shape:', train_y.shape)
print('val_y.shape: ',val_y.shape)





tweet=all_positive_tweets[0]
tokens=process_tweet(tweet)
print(tokens)

number of positive tweets 5000
number of negative tweets 5000
number of training tweets 8000
number of validation tweets 2000
train_y.shape: (8000,)
val_y.shape:  (2000,)
['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


In [ ]:


Vocab = {'__PAD__': 0, '__</e>__': 1, '__UNK__': 2}

for tweet in train_x:

    tokens=process_tweet(tweet)

    for word in tokens:

        if word not in Vocab:

            Vocab[word]=len(Vocab)

print(Vocab)

print('vocab size',len(Vocab))



def tweet_to_tensor(tweet, vocab_dict, unk_token='__UNK__', verbose=False):


    tweet=process_tweet(tweet)
    tensor=[]

    if verbose:
        print("The unique integer ID for the unk_token is {}".format(unk_token))
    for word in tweet:


        if word in vocab_dict:

            tensor.append(vocab_dict[word])

        else:

            tensor.append(vocab_dict[unk_token])

    return tensor





print('actual Tweet :\n',val_x[0])
print('tensor is \n',tweet_to_tensor(val_x[0],vocab_dict=Vocab))


{'__PAD__': 0, '__</e>__': 1, '__UNK__': 2, 'followfriday': 3, 'top': 4, 'engag': 5, 'member': 6, 'commun': 7, 'week': 8, ':)': 9, 'hey': 10, 'jame': 11, 'odd': 12, ':/': 13, 'pleas': 14, 'call': 15, 'contact': 16, 'centr': 17, '02392441234': 18, 'abl': 19, 'assist': 20, 'mani': 21, 'thank': 22, 'listen': 23, 'last': 24, 'night': 25, 'bleed': 26, 'amaz': 27, 'track': 28, 'scotland': 29, 'congrat': 30, 'yeaaah': 31, 'yipppi': 32, 'accnt': 33, 'verifi': 34, 'rqst': 35, 'succeed': 36, 'got': 37, 'blue': 38, 'tick': 39, 'mark': 40, 'fb': 41, 'profil': 42, '15': 43, 'day': 44, 'one': 45, 'irresist': 46, 'flipkartfashionfriday': 47, 'like': 48, 'keep': 49, 'love': 50, 'custom': 51, 'wait': 52, 'long': 53, 'hope': 54, 'enjoy': 55, 'happi': 56, 'friday': 57, 'lwwf': 58, 'second': 59, 'thought': 60, '’': 61, 'enough': 62, 'time': 63, 'dd': 64, 'new': 65, 'short': 66, 'enter': 67, 'system': 68, 'sheep': 69, 'must': 70, 'buy': 71, 'jgh': 72, 'go': 73, 'bayan': 74, ':D': 75, 'bye': 76, 'act': 77, 

In [ ]:
def generator_data(data_x,data_y,batch_size,Vocab):


    number_of_samples=len(data_x)
    data_y=data_y.tolist()
    while 1:


        data_x,data_y=shuffle(data_x,data_y)


        for i in range(0,number_of_samples,batch_size):


            x_out=data_x[i:i+batch_size]

            y_out=data_y[i:i + batch_size]


            tensor_l=[]

            for tweet in x_out:

                tensor_l.append(tweet_to_tensor(tweet, Vocab, unk_token='__UNK__', verbose=False))


            ### zero pad to the longest tweet

            tensor_l_padded=[]

            max_length=max([len(x) for x in tensor_l])

            for tensor in tensor_l:



                tensor_l_padded.append(tensor+[0]*(max_length-len(tensor)))


            tensor_l_padded=np.array(tensor_l_padded)


            yield (tensor_l_padded,np.array(y_out))


In [ ]:
## train and test generators
batch_size=16
generator_train=generator_data(train_x,train_y,batch_size=batch_size,Vocab=Vocab)  #train generator

batch_x,batch_y=next(generator_train)

print(batch_x.shape)
print(batch_y.shape)


generator_val=generator_data(val_x,val_y,batch_size=batch_size,Vocab=Vocab)  #val generator

batch_x,batch_y=next(generator_val)

print(batch_x.shape)
print(batch_y.shape)
#print(batch_y)

(16, 14)
(16,)
(16, 18)
(16,)


In [ ]:
Input=tf.keras.layers.Input(shape=(None,))
embed=tf.keras.layers.Embedding(input_dim=len(Vocab),output_dim=256)(Input)
#print(embed)
embed=tf.reduce_mean(embed,axis=1)
#print(embed)
dense=tf.keras.layers.Dense(units=64,activation=tf.nn.relu)(embed)
pred=tf.keras.layers.Dense(units=1,activation=tf.nn.sigmoid)(dense)

model=tf.keras.Model(inputs=Input,outputs=pred)

print(model.summary())








Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_8 (Embedding)      (None, None, 256)         2327552   
_________________________________________________________________
tf.math.reduce_mean_4 (TFOpL (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 2,344,065
Trainable params: 2,344,065
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),metrics=[tf.keras.metrics.BinaryAccuracy()])


In [ ]:
model.fit(generator_train,validation_data=generator_val,epochs=3,verbose=1,steps_per_epoch=len(train_x)/batch_size,validation_steps=len(val_x)/batch_size)


Epoch 1/3
500/500 [==============================] - 16s 31ms/step - loss: 0.1339 - binary_accuracy: 0.9679 - val_loss: 0.0141 - val_binary_accuracy: 0.9950
Epoch 2/3
500/500 [==============================] - 15s 30ms/step - loss: 0.0112 - binary_accuracy: 0.9966 - val_loss: 0.0070 - val_binary_accuracy: 0.9975
Epoch 3/3
500/500 [==============================] - 16s 31ms/step - loss: 0.0051 - binary_accuracy: 0.9985 - val_loss: 0.0139 - val_binary_accuracy: 0.9950


In [ ]:

def predict_sentiment(test_tweet,vocab_dict):

  tensor=tweet_to_tensor(test_tweet,vocab_dict=vocab_dict)

  tensor=np.array(tensor).reshape(1,-1)

  #print(tensor.shape)

  prediction=model.predict(tensor)

  #print(prediction)

  print('input tweet is:',test_tweet)
  if prediction>0.5:

    print('the tweet is positive with confidence of {}'.format(prediction.item()))

  else:

    print('the tweet is negative with confidence of {}'.format(prediction.item()))



In [ ]:
test_tweet='I am happy today because I talked to my mother.'
predict_sentiment(test_tweet,vocab_dict=Vocab)

input tweet is: I am happy today because I talked to my mother.
the tweet is positive with confidence of 0.9094496965408325


In [ ]:
test_tweet='Sadly They elected a dictator as thier leader.'
predict_sentiment(test_tweet,vocab_dict=Vocab)

input tweet is: Sadly They elected a dictator as thier leader.
the tweet is negative with confidence of 0.3456490635871887


In [ ]:

test_tweet='Was lucky enough to play yet another festival at the weekend down under here in Australia... 7,000 people. No covid restrictions... got the crowd to sing happy birthday to my wife. Performed my new single Smiling faceZany face Beyond grateful'

predict_sentiment(test_tweet,vocab_dict=Vocab)

input tweet is: Was lucky enough to play yet another festival at the weekend down under here in Australia... 7,000 people. No covid restrictions... got the crowd to sing happy birthday to my wife. Performed my new single Smiling faceZany face Beyond grateful
the tweet is positive with confidence of 0.7941251993179321
